# MED_ANTIP_ANY_PRI is actually a surrogate for any mental illness

- out of 509 SZP, BPP, SADP patients, 423 of them are on this med; assume no NC or ...R are on this med
- Actually a total of 471 are on the med

- 73% clf accuracy for NC versus med takers
- 74% clf accuracy for NC versus patients

- MED_PSYCHOTROPIC is also a surrogate
- MED_ANTIP_SECONDGEN_SEC is a subcategory that's also a good surrogate

In [2]:
import pickle
from natsort import natsorted

basedir = '../../ImageNomer/data/anton/cohorts/BSNIP'
demoname = f'{basedir}/demographics.pkl'

with open(demoname, 'rb') as f:
    demo = pickle.load(f)
    
subs = natsorted(list(demo['MED_ANTIP_ANY_PRI'].keys()))
print(len(subs))

1224


In [43]:
import numpy as np

ox = []
oy = []

for sub in subs:
#     if demo['DXGROUP_2'][sub] in ['SZP', 'SADP', 'BPP']:
# #     if demo['MED_PSYCHOTROPIC'][sub] == 'YES':
#         oy.append(1)
#     elif demo['DXGROUP_2'][sub] == 'NC':
#         oy.append(0)
#     else:
#         continue
    if sub not in demo['Age_cal']:
        continue
    p = np.load(f'{basedir}/fc/{sub}_task-unk_fc.npy')
    ox.append(p)
    oy.append(int(demo['Age_cal'][sub]))
#     oy.append(demo['MED_ANTIP_ANY_PRI'][sub] == 'YES')
    
ox = np.stack(ox)
oy = np.array(oy).astype('int')

print(ox.shape)
print(oy.shape)
print(oy[0:5])

(1222, 34716)
(1222,)
[23 53 52 51 47]


In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import sys

if '../../LatentSimilarity' not in sys.path:
    sys.path.append('../../LatentSimilarity')
    
from latsim import LatSimClf, LatSimReg
accs = []

for _ in range(10):
    xtr, xt, ytr, yt = train_test_split(ox, oy, stratify=oy, train_size=0.6)

#     clf = LogisticRegression(C=1, max_iter=1000).fit(xtr, ytr)
    clf = LatSimReg().fit(xtr, ytr, wd=1e-3, ld=1, nepochs=100)
    yhat = clf.predict(xt)
#     acc = np.mean((yhat == yt).astype('int'))
    acc = np.mean((yhat-yt)**2)**0.5
    acc = float(acc)
    print(acc)
    accs.append(acc)
    
print('---')
print(np.mean(accs))
print(np.std(accs))

11.023744628138935
10.61150331181143
10.629767646353846
10.910980999058175
11.388998885767759
11.131760302461704
11.827496660489208
11.121361490047171
11.304651901150715
11.005624465479043
---
11.0955890290758
0.34148527022768405
